# Introdução à Otimização de Hiperparâmetros

Leão Pereira Neto

R.A: 22200115


In [12]:
from sklearn.linear_model import LogisticRegression

# Criando um modelo de Regressão Logística
modelo = LogisticRegression()

In [28]:
# Carregando dataset do sklearn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, InputLayer
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go

#Carregando o dataset de dígitos
data = pd.read_csv('notebook.csv')


In [29]:
data.head()

,marca,ano,qtd_nucleos,qtd_threads,ram,gpu_dedicada,ram_gpu,entradas_usb,duracao_bateria,resolucao_tela,tipo_tela,tela_touch,armazenamento_hdd,armazenamento_ssd,segmento,valor
0,xiaomi,2012,12,19,16,não,0,2,434,FULLHD,OLED,não,0,2000,premium,18215
1,dell,2014,6,8,6,não,0,2,566,FULLHD,TN,não,500,1000,ultraportatil,18904
2,asus,2022,6,12,12,não,0,3,281,FULLHD,AMOLED,sim,0,500,2em1,14217
3,xiaomi,2014,8,9,16,não,0,3,473,QUADHD,OLED,não,2000,250,premium,19607
4,asus,2023,10,10,8,não,0,3,506,QUADHD,OLED,não,0,1000,premium,19841


In [30]:
data.describe()

,ano,qtd_nucleos,qtd_threads,ram,ram_gpu,entradas_usb,duracao_bateria,armazenamento_hdd,armazenamento_ssd,valor
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,2016.425000,7.347400,10.996000,13.881600,1.237800,2.605800,346.455000,333.625000,622.850000,15311.950000
std,4.005194,2.930558,5.057108,10.764644,2.369306,0.886727,191.551615,573.559941,649.398798,6060.214458
min,2010.000000,2.000000,2.000000,2.000000,0.000000,1.000000,30.000000,0.000000,0.000000,2599.000000
25%,2013.000000,4.000000,7.000000,8.000000,0.000000,2.000000,151.000000,0.000000,250.000000,9836.750000
50%,2016.000000,8.000000,10.000000,12.000000,0.000000,3.000000,419.000000,0.000000,250.000000,16078.000000
75%,2020.000000,10.000000,14.000000,16.000000,0.000000,3.000000,523.000000,500.000000,1000.000000,20189.250000
max,2023.000000,12.000000,24.000000,64.000000,12.000000,4.000000,600.000000,2000.000000,2000.000000,27844.000000


In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   marca              10000 non-null  object
 1   ano                10000 non-null  int64 
 2   qtd_nucleos        10000 non-null  int64 
 3   qtd_threads        10000 non-null  int64 
 4   ram                10000 non-null  int64 
 5   gpu_dedicada       10000 non-null  object
 6   ram_gpu            10000 non-null  int64 
 7   entradas_usb       10000 non-null  int64 
 8   duracao_bateria    10000 non-null  int64 
 9   resolucao_tela     10000 non-null  object
 10  tipo_tela          10000 non-null  object
 11  tela_touch         10000 non-null  object
 12  armazenamento_hdd  10000 non-null  int64 
 13  armazenamento_ssd  10000 non-null  int64 
 14  segmento           10000 non-null  object
 15  valor              10000 non-null  int64 
dtypes: int64(10), object(6)
memory usage: 1.2

In [32]:
data.isna().sum() *100/len(data)

marca                0.0
ano                  0.0
qtd_nucleos          0.0
qtd_threads          0.0
ram                  0.0
gpu_dedicada         0.0
ram_gpu              0.0
entradas_usb         0.0
duracao_bateria      0.0
resolucao_tela       0.0
tipo_tela            0.0
tela_touch           0.0
armazenamento_hdd    0.0
armazenamento_ssd    0.0
segmento             0.0
valor                0.0
dtype: float64

In [33]:
# Cria uma lista com todas as colunas do tipo object
colunas_object  = data.select_dtypes(include='object').columns.tolist()

# Exibe a lista com todas as colunas do tipo object
colunas_object

['marca',
 'gpu_dedicada',
 'resolucao_tela',
 'tipo_tela',
 'tela_touch',
 'segmento']

In [34]:
for coluna in colunas_object:
    le = LabelEncoder()
    data[coluna] = le.fit_transform(data[coluna])

In [24]:
X = data[data.columns[:-1]]
y= data[['valor']]

In [35]:
X.shape

(10000, 15)

In [36]:
y.shape

(10000, 1)

In [37]:
np.unique(y)

array([ 2599,  2846,  2897, ..., 27747, 27781, 27844], dtype=int64)

In [39]:
# Dividindo em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando uma rede neural simples
#model = Sequential()
#model.add(Dense(4, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dense(2, activation='relu'))
#model.add(Dense(64, activation='relu'))
#model.add(Dense(10, activation='softmax'))

def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(4, input_dim=X_train.shape[1], activation=activation)) # Camada oculta com a função de ativação variável
    model.add(Dense(2, activation=activation)) # Camada de saída
    model.add(Dense(64, activation=activation))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy']) # Compilação do modelo com MSE como função de perda
    return model

# Compilando e treinando o modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

# Criação do modelo KerasRegressor para o uso com o GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)
# O KerasRegressor vai permirtir que o GfridSearchCV controle o processo de treinamento
# Uma vez que o GridSearchCV orecisa treinar vários modelos com diferentes combinações, o KEras vai envovler cada modelo de treinamento

# Parâmetros a serem testados
param_grid = {
    'activation': ['relu', 'tanh', 'sigmoid']
}

# GridSearchCV para encontrar a melhor função de ativação
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Resultados do GridSearchCV
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor MSE: ", -grid_result.best_score_)

C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\1856298185.py:27: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11

Melhores parâmetros:  {'activation': 'relu'}
Melhor MSE:  nan


In [43]:
# Dividindo em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função para criar o modelo
def create_model(neurons_layer1=64, neurons_layer2=64):
    model = Sequential()
    model.add(Dense(neurons_layer1, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(neurons_layer2, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

# Criação do modelo KerasRegressor para o uso com o GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Parâmetros a serem testados
param_grid = {
    'neurons_layer1': [32, 64, 128],
    'neurons_layer2': [32, 64, 128]
}

# GridSearchCV para encontrar a melhor combinação de neurônios
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Resultados do GridSearchCV
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor MSE: ", -grid_result.best_score_)

C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\114152046.py:14: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_

Melhores parâmetros:  {'neurons_layer1': 32, 'neurons_layer2': 32}
Melhor MSE:  nan


In [44]:
# Dividindo em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função para criar o modelo
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])
    return model

# Criação do modelo KerasRegressor para o uso com o GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Parâmetros a serem testados
param_grid = {
    'optimizer': ['adam', 'sgd', 'rmsprop']
}

# GridSearchCV para encontrar o melhor otimizador
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Resultados do GridSearchCV
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor MSE: ", -grid_result.best_score_)

C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\1871082651.py:14: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11

Melhores parâmetros:  {'optimizer': 'adam'}
Melhor MSE:  nan


In [46]:
# Função para criar o modelo
def create_model(batch_size=32, epochs=10):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

# Criação do modelo KerasRegressor para o uso com o GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Parâmetros a serem testados
param_grid = {
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

# GridSearchCV para encontrar o melhor batch_size e epochs
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Resultados do GridSearchCV
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor MSE: ", -grid_result.best_score_)

C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\1802829466.py:11: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11

Melhores parâmetros:  {'batch_size': 32, 'epochs': 10}
Melhor MSE:  nan


In [47]:
from keras.regularizers import l2

# Função para criar o modelo
def create_model(regularization=l2(0.01)):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=regularization))
    model.add(Dense(32, activation='relu', kernel_regularizer=regularization))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

# Criação do modelo KerasRegressor para o uso com o GridSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Parâmetros a serem testados
param_grid = {
    'regularization': [l2(0.01), l2(0.001), l2(0.0001)]
}

# GridSearchCV para encontrar o melhor valor para a regularização L2
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3)
grid_result = grid.fit(X_train, y_train)

# Resultados do GridSearchCV
print("Melhores parâmetros: ", grid_result.best_params_)
print("Melhor MSE: ", -grid_result.best_score_)

C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\1276358587.py:13: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11

Melhores parâmetros:  {'regularization': <keras.regularizers.L2 object at 0x000001EACA3CA010>}
Melhor MSE:  nan


In [42]:
# Definindo hiperparâmetros para uma rede neural
learning_rate = 0.001
batch_size = 32
epochs = 50
number_of_neurons_layer1 = 128
number_of_neurons_layer2 = 64
number_of_neurons_layer3 = 128
activation_function = 'relu'

# Criando um modelo usando os hiperparâmetros
model = Sequential()
model.add(Dense(number_of_neurons_layer1, input_dim=X_train.shape[1], activation=activation_function))
model.add(Dense(number_of_neurons_layer2, activation=activation_function))
model.add(Dense(number_of_neurons_layer3, activation=activation_function))
model.add(Dense(10, activation='softmax'))

# Treinando o modelo usando os hiperparâmetros
model.compile(loss='MSE', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)


# Randonsearch

In [49]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

# Dividindo em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Função para criar o modelo
def create_model(activation='relu'):
    model = Sequential()
    model.add(Dense(32, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(32, activation=activation))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    return model

# Criação do modelo KerasRegressor para o uso com o RandomizedSearchCV
model = KerasRegressor(build_fn=create_model, verbose=0)

# Parâmetros a serem testados aleatoriamente
param_dist = {
    'activation': ['relu', 'tanh', 'sigmoid']
}

# RandomizedSearchCV para encontrar aleatoriamente a melhor função de ativação
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=10, scoring='neg_mean_squared_error', cv=3)
random_search_result = random_search.fit(X_train, y_train)

# Resultados do RandomizedSearchCV
print("Melhores parâmetros: ", random_search_result.best_params_)
print("Melhor MSE: ", -random_search_result.best_score_)


C:\Users\Leão Neto\AppData\Local\Temp\ipykernel_24548\294261784.py:21: DeprecationWarning:

KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_search.py:305: UserWarning:

The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.

C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py:778: UserWarning:

Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Leão Neto\AppData\Local\Packages\PythonSoftwareFoundat

Melhores parâmetros:  {'activation': 'relu'}
Melhor MSE:  nan


# parte do prof

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def metricas_classificacao(y_real, y_pred):
    # Calcular métricas
    metrics = {
        "Acurácia": accuracy_score(y_real, y_pred),
        "Precisão (macro)": precision_score(y_real, y_pred, average='macro'),
        "Recall (macro)": recall_score(y_real, y_pred, average='macro'),
        "F1-Score (macro)": f1_score(y_real, y_pred, average='macro')
    }
    
    # Printar métricas
    for key, value in metrics.items():
        print(f"{key}: {value}")

    # Calcular a Matriz de Confusão
    confusion_mat = confusion_matrix(y_real, y_pred)

    # Printar Matriz de Confusão
    print("Matriz de Confusão:")
    sns.heatmap(confusion_mat, annot=True, cmap='YlGnBu', fmt='g')
    plt.xlabel('Previsto')
    plt.ylabel('Real')
    plt.show()

    return metrics


In [ ]:
# Importando as bibliotecas necessárias para construir o modelo da rede neural
from keras.models import Sequential
from keras.layers import Dense

# Função que cria e retorna um modelo de rede neural
def create_model(neurons1=128, neurons2=64, activation_function='relu'):
    
    # Inicializando o modelo sequencial
    model = Sequential()
    
    # Adicionando a primeira camada oculta com o número de neurônios especificado (neurons1)
    # 'input_dim' especifica o número de características de entrada
    model.add(Dense(neurons1, input_dim=X_train.shape[1], activation=activation_function))
    
    # Adicionando a segunda camada oculta com o número de neurônios especificado (neurons2)
    model.add(Dense(neurons2, activation=activation_function))
    
    # Adicionando a terceira camada oculta com 128 neurônios
    model.add(Dense(128, activation=activation_function))
    
    # Adicionando a camada de saída com 10 neurônios (para um problema de classificação de 10 classes)
    model.add(Dense(10, activation='softmax'))
    
    # Compilando o modelo - especificando o otimizador, a função de perda e a métrica de avaliação
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Importando o KerasClassifier que permite usar modelos Keras com scikit-learn
from keras.wrappers.scikit_learn import KerasClassifier
# Importando GridSearchCV para otimização de hiperparâmetros
from sklearn.model_selection import GridSearchCV

# Criando uma instância do KerasClassifier com a função de criação do modelo e especificando 20 épocas
model_for_grid = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

# Definindo a grade de hiperparâmetros que queremos testar
param_grid = {
    'neurons1': [64, 128, 256],                # diferentes quantidades de neurônios para a primeira camada
    'neurons2': [64, 128, 256],                # diferentes quantidades de neurônios para a segunda camada
    'activation_function': ['relu', 'tanh']  # diferentes funções de ativação
}

# Configurando o GridSearchCV com o modelo, a grade de parâmetros, e outras opções
grid = GridSearchCV(estimator=model_for_grid, param_grid=param_grid, n_jobs=-1, cv=3)

# Iniciando a busca pelos melhores hiperparâmetros usando o conjunto de treinamento
grid_result = grid.fit(X_train, y_train)

# Imprimindo os melhores hiperparâmetros encontrados
print(f"Melhores parâmetros: {grid_result.best_params_}")

In [ ]:
3 * 3 * 2 * 20

In [ ]:
best_model = create_model(neurons1=128, neurons2=128, activation_function='relu')
best_model.fit(X_train, y_train, epochs=20, verbose = 0)
y_pred = np.argmax(best_model.predict(X_test), axis=1)
metricas = metricas_classificacao(y_test, y_pred)

In [ ]:
model = create_model(neurons1=64, neurons2=64, activation_function='relu')
model.fit(X_train, y_train, epochs=20, verbose = 0)
y_pred = np.argmax(model.predict(X_test), axis=1)
metricas = metricas_classificacao(y_test, y_pred)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Definindo o espaço de hiperparâmetros
param_dist = {
    'neurons1': [64, 128, 256, 512],
    'neurons2': [64, 128, 256, 512],
    'activation_function': ['relu', 'tanh', 'softmax']
}

model_for_grid = KerasClassifier(build_fn=create_model, epochs=20, verbose=0)

# Configurando o RandomizedSearchCV com 10 iterações
random_search = RandomizedSearchCV(estimator=model_for_grid, param_distributions=param_dist, n_iter=10, n_jobs=-1, cv=3)

# Iniciando a busca pelos melhores hiperparâmetros usando o conjunto de treinamento
random_result = random_search.fit(X_train, y_train)

# Imprimindo os melhores hiperparâmetros encontrados
print(f"Melhores parâmetros usando RandomizedSearchCV: {random_result.best_params_}")


In [ ]:
model = create_model(neurons1=128, neurons2=256, activation_function='relu')
model.fit(X_train, y_train, epochs=20, verbose = 0)
y_pred = np.argmax(model.predict(X_test), axis=1)
metricas = metricas_classificacao(y_test, y_pred)

In [ ]:
!pip install bayesian-optimization

In [ ]:
# Importando a biblioteca necessária
from bayes_opt import BayesianOptimization

# Definindo a função objetivo que queremos otimizar.
# Esta função treina um modelo com hiperparâmetros fornecidos e retorna a acurácia de validação.
def objective_function(neurons1, neurons2, activation_index):
    
    # Mapeia os índices para suas respectivas funções de ativação.
    activation_functions = ['relu', 'tanh']
    activation_function = activation_functions[int(activation_index)]
    
    # Cria e compila o modelo usando os hiperparâmetros fornecidos.
    model = create_model(int(neurons1), int(neurons2), activation_function)
    
    # Treina o modelo e obtém o histórico de treinamento.
    history = model.fit(X_train, y_train, epochs=20, verbose=0, validation_split=0.2)
    
    # Obtém a acurácia de validação da última época.
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy

# Definindo os limites dos hiperparâmetros para a otimização bayesiana.
# Os hiperparâmetros contínuos têm intervalos definidos (por exemplo, neurons1 entre 64 e 256).
# Para hiperparâmetros categóricos, usamos índices (por exemplo, activation_index entre 0 e 1).
pbounds = {
    'neurons1': (64, 256),
    'neurons2': (64, 256),
    'activation_index': (0, 1)
}

# Inicializa o otimizador bayesiano com a função objetivo e os limites dos hiperparâmetros.
optimizer = BayesianOptimization(
    f=objective_function,     # Função objetivo definida anteriormente.
    pbounds=pbounds,          # Limites dos hiperparâmetros.
    random_state=1            # Semente para reprodutibilidade.
)

# Executa a otimização bayesiana.
# 'init_points' define quantos pontos iniciais aleatórios serão testados antes da otimização começar.
# 'n_iter' define quantas iterações de otimização serão executadas.
optimizer.maximize(init_points=5, n_iter=10)

# Exibe os melhores hiperparâmetros encontrados.
print(f"Melhores parâmetros usando otimização bayesiana: {optimizer.max['params']}")